In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency

# Importar datos desde un csv con pandas 

In [2]:
loan_data = pd.read_csv('C:/Users/diego/OneDrive/Escritorio/prestamos_2015.csv')

C:\Users\diego\AppData\Local\Temp\ipykernel_6692\1906435946.py:1: DtypeWarning: Columns (19,47,55) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv('C:/Users/diego/OneDrive/Escritorio/prestamos_2015.csv')


# Exploración de los datos

In [3]:
pd.options.display.max_columns = None
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,60516983,64537751,20000,20000,20000,36 months,12.29,667.06,C,C1,Accounting Clerk,1 year,OWN,65000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,542xx,WI,20.72,0,Sep-00,1,NaN,NaN,25,0,31578,77.0,42,w,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,NaN,0.00,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,52303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41000,NaN,NaN,NaN
1,60187139,64163931,11000,11000,11000,36 months,12.69,369.00,C,C2,Accounts Payable Lead,7 years,MORTGAGE,40000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,235xx,VA,24.57,0,Sep-02,0,36.0,80.0,13,1,5084,38.8,41,w,0.0,0.0,10043.49,10043.49,9942.67,100.81,0.0,0.0,0.0,Oct-15,10059.00,NaN,Jan-16,0,79.0,1,INDIVIDUAL,NaN,NaN,NaN,0,332,175731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13100,NaN,NaN,NaN
2,60356453,64333218,7000,7000,7000,36 months,9.99,225.84,B,B3,Nurse,6 years,MORTGAGE,32000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,350xx,AL,32.41,0,Feb-06,1,NaN,NaN,18,0,12070,74.0,36,f,0.0,0.0,221.96,221.96,167.56,54.40,0.0,0.0,0.0,Oct-15,225.84,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,202012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16300,NaN,NaN,NaN
3,59955769,63900496,10000,10000,10000,36 months,10.99,327.34,B,B4,Service Manager,10+ years,MORTGAGE,48000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,483xx,MI,30.98,0,Oct-99,2,NaN,NaN,18,0,22950,66.0,41,f,0.0,0.0,315.13,315.13,235.76,79.37,0.0,0.0,0.0,Oct-15,327.34,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,108235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34750,NaN,NaN,NaN
4,58703693,62544456,9550,9550,9550,36 months,19.99,354.87,E,E4,NaN,NaN,RENT,32376.0,Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,546xx,WI,32.54,0,Nov-99,3,69.0,NaN,9,0,4172,29.6,26,w,0.0,0.0,333.66,333.66,195.78,137.88,0.0,0.0,0.0,Oct-15,354.87,NaN,Jan-16,0,69.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,45492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14100,NaN,NaN,NaN


In [4]:
loan_data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,4.210940e+05,4.210940e+05,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,4.210940e+05,421094.000000,421094.000000,421094.000000,217133.000000,74415.000000,421094.000000,421094.000000,4.210940e+05,420932.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,421094.000000,122729.000000,421094.0,511.000000,509.000000,421094.000000,421094.000000,4.210940e+05,21372.000000,21372.000000,21372.000000,21372.000000,20810.000000,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,4.210940e+05,21372.000000,21372.000000,21372.000000
mean,5.393094e+07,5.759572e+07,15240.255561,15240.255561,15234.125991,12.601731,441.872719,7.696561e+04,19.196014,0.347397,0.572692,34.023391,66.592609,11.948629,0.233777,1.768548e+04,53.839819,25.493458,12824.715167,12819.308696,3149.641624,3148.657478,2315.820515,831.441789,0.116056,2.263264,0.362040,1101.376892,0.020240,45.114741,1.0,109981.011585,18.310118,0.006087,257.478853,1.400756e+05,1.109021,2.928832,0.761651,1.674574,20.912686,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.365751e+04,0.943945,1.537058,1.977307
std,9.475416e+06,1.013357e+07,8571.313525,8571.313525,8567.923312,4.317873,244.857549,7.395005e+04,23.504060,0.927696,0.867777,21.990270,25.577899,5.633118,0.649778,2.419506e+04,23.886467,12.097212,8372.087005,8368.291516,4232.228229,4231.088416,3899.732266,893.458450,2.088179,111.428440,16.286269,3482.997210,0.157506,22.537808,0.0,52730.379847,7.169233,0.086482,2240.650788,1.552698e+05,1.242675,3.089987,0.996035,1.688725,27.209081,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.766386e+04,1.446872,2.723339,2.874067
min,5.670500e+04,7.082500e+04,1000.000000,1000.000000,900.000000,5.320000,30.120000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,17950.000000,3.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,4.612481e+07,4.922616e+07,8500.000000,8500.000000,8500.000000,9.170000,263.930000,4.600000e+04,12.590000,0.000000,0.000000,15.000000,50.000000,8.000000,0.000000,6.480000e+03,36.100000,17.000000,6457.280000,6452.190000,854.252500,853.680000,550.460000,224.720000,0.000000,0.000000,0.000000,256.300000,0.000000,27.000000,1.0,76032.500000,13.200000,0.000000,0.000000,3.103900e+04,0.000000,1.000000,0.000000,0.000000,6.000000,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.440000e+04,0.000000,0.000000,0.000000
50%,5.536030e+07,5.895358e+07,14000.000000,14000.000000,14000.000000,12.290000,385.425000,6.500000e+04,18.600000,0.000000,0.000000,31.000000,67.000000,11.000000,0.000000,1.199600e+04,54.200000,24.000000,11444.610000,11440.480000,1971.110000,1970.600000,1243.730000,537.530000,0.000000,0.000000,0.000000,392.810000,0.000000,45.000000,1.0,101771.000000,17.610000,0.000000,0.000000,7.972900e+04,1.000000,2.000000,0.000000,1.000000,12.000000,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.460000e+04,0.000000,0.000

In [5]:
#tipo de dato en cada columna
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421094 entries, 0 to 421093
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           421094 non-null  int64  
 1   member_id                    421094 non-null  int64  
 2   loan_amnt                    421094 non-null  int64  
 3   funded_amnt                  421094 non-null  int64  
 4   funded_amnt_inv              421094 non-null  int64  
 5   term                         421094 non-null  object 
 6   int_rate                     421094 non-null  float64
 7   installment                  421094 non-null  float64
 8   grade                        421094 non-null  object 
 9   sub_grade                    421094 non-null  object 
 10  emp_title                    397220 non-null  object 
 11  emp_length                   397277 non-null  object 
 12  home_ownership               421094 non-null  object 
 13 

In [6]:
#Mostramos de manera ordena el % de valores nulos en cada columna 
for col in loan_data:
    missing = round(loan_data[col].isna().sum() / len(loan_data) * 100,2)
    print(f"Total de valores faltantes en {col} feature: {missing} %")

Total de valores faltantes en id feature: 0.0 %
Total de valores faltantes en member_id feature: 0.0 %
Total de valores faltantes en loan_amnt feature: 0.0 %
Total de valores faltantes en funded_amnt feature: 0.0 %
Total de valores faltantes en funded_amnt_inv feature: 0.0 %
Total de valores faltantes en term feature: 0.0 %
Total de valores faltantes en int_rate feature: 0.0 %
Total de valores faltantes en installment feature: 0.0 %
Total de valores faltantes en grade feature: 0.0 %
Total de valores faltantes en sub_grade feature: 0.0 %
Total de valores faltantes en emp_title feature: 5.67 %
Total de valores faltantes en emp_length feature: 5.66 %
Total de valores faltantes en home_ownership feature: 0.0 %
Total de valores faltantes en annual_inc feature: 0.0 %
Total de valores faltantes en verification_status feature: 0.0 %
Total de valores faltantes en issue_d feature: 0.0 %
Total de valores faltantes en loan_status feature: 0.0 %
Total de valores faltantes en pymnt_plan feature: 0.0

In [7]:
# obtiene las columnas que tenga mas de 75% de datos nulos
na_values = loan_data.isnull().mean()
na_values[na_values>0.75]

desc                         0.999893
mths_since_last_record       0.823282
annual_inc_joint             0.998786
dti_joint                    0.998791
verification_status_joint    0.998786
open_acc_6m                  0.949246
open_il_6m                   0.949246
open_il_12m                  0.949246
open_il_24m                  0.949246
mths_since_rcnt_il           0.950581
total_bal_il                 0.949246
il_util                      0.955789
open_rv_12m                  0.949246
open_rv_24m                  0.949246
max_bal_bc                   0.949246
all_util                     0.949246
inq_fi                       0.949246
total_cu_tl                  0.949246
inq_last_12m                 0.949246
dtype: float64

# Limpieza inicial de datos 

In [8]:
# elimina las columnas con ams de 75% de datos nulos
loan_data.dropna(thresh = loan_data.shape[0]*0.25, axis = 1, inplace = True)

'''
Vamos a eliminar  aquellas columnas redundantes y que no aportan información util , estadisticamente hablando.
[id, member_id, title, emp_title,zip_code, url]

también suprimimos las columnas numéricas sin relación con la probabilidad de impago
[next_pymnt_d, recoveries, collection_recovery_fee, total_rec_prncp, total_rec_late_fee]

Eliminamos las variables 'grade' y 'subgrade', ya que el grado de calidad de un crédito está directamente relacionado 
con la Probabilidad de Incumplimiento (PD). Si buscamos desarrollar un modelo de riesgo crediticio desde cero, 
no contaríamos con información sobre la calidad del crédito.
'''

loan_data.drop(columns = ['id', 'member_id', 'grade','sub_grade', 'emp_title', 'url', 'title', 'zip_code', 'next_pymnt_d',
                          'recoveries', 'collection_recovery_fee', 'total_rec_prncp', 'total_rec_late_fee'], inplace = True)

In [9]:
# Re-imprimimos un resumen 
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421094 entries, 0 to 421093
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    421094 non-null  int64  
 1   funded_amnt                  421094 non-null  int64  
 2   funded_amnt_inv              421094 non-null  int64  
 3   term                         421094 non-null  object 
 4   int_rate                     421094 non-null  float64
 5   installment                  421094 non-null  float64
 6   emp_length                   397277 non-null  object 
 7   home_ownership               421094 non-null  object 
 8   annual_inc                   421094 non-null  float64
 9   verification_status          421094 non-null  object 
 10  issue_d                      421094 non-null  object 
 11  loan_status                  421094 non-null  object 
 12  pymnt_plan                   421094 non-null  object 
 13 

# Crear la columna objetivo

In [10]:

'''
Vamos a definir el significado de cada tipo de estatutos del préstamo:

Fully Paid: El préstamo ha sido pagado por completo por el prestatario.

Charged Off: El préstamo a sido declarado como perdido y dado de baja, pasa a formar parte de NPL (Non-performing loan), según el
            la institución este periodo puede variar entre 90 y 180 días. 
                              

Default: El préstamo ha entrado en incumplimiento, es decir, el prestatario no ha pagado en al menos 120 días.

Late (16-30 days): El préstamo tiene un pago pendiente de 16 a 30 días.

Late (31-120 days): El préstamo tiene un pago pendiente de 31 a 120 días.

In Grace Period: significa que el prestatario ha retrasado su pago entre 1 y 15 días,
                 esto entra dentro de los préstamos no arriesgados, pero si el prestatario no paga, se convierte en un préstamo de riesgo.

Current: El préstamo está al día, pero aun cuenta con pagos a futuro pendientes.

issued: El préstamo ha sido recién aprobado, se encuentra por iniciar el ciclo  de pagos.


'''
# Revisamos la distribución de la variable objetivo
loan_data['loan_status'].value_counts(normalize = True)

loan_status
Current               0.896600
Fully Paid            0.054582
Issued                0.020091
Late (31-120 days)    0.011140
In Grace Period       0.007378
Charged Off           0.006585
Late (16-30 days)     0.002705
Default               0.000919
Name: proportion, dtype: float64

In [11]:
'''
Vamos a definir las categorías de riesgo de la siguiente manera:

Prestamos de riesgo 
    - Default
    - Late (31-120 days)
    - In Grace Period
    - Late (16-30 days)
    - Charged Off
    
Prestamos susceptibles a riesgo
    - Current
    - issued
    - In Grace Period
    
Prestamos libre de riesgo
    - Fully Paid
    
    
Para el análisis se considerará:

1 (default): Default, Late (31-120 days), Late (16-30 days), Charged Off.
0 (no default): Current, Issued, Fully Paid.

se elimino la categoría In Grace Period, debido a su naturaleza de transición entre prestamos de riesgo y préstamo susceptible a riesgo.
'''


loan_data= loan_data[ loan_data['loan_status'] != 'In Grace Period']

loan_data['target'] = np.where(loan_data['loan_status'].isin(['Default', 'Late (31-120 days)', 'Late (16-30 days)', 'Charged Off']), 1, 0)

#eliminamos la columna loan_status
loan_data.drop( columns = ['loan_status'], inplace = True)


### Revisamos las variables categóricas y verificamos si hay que hacer alguna transformación        

In [12]:
cat_cols = loan_data.select_dtypes('object').columns

for col in cat_cols: print(col, loan_data[col].nunique())



term 2
emp_length 11
home_ownership 4
verification_status 3
issue_d 12
pymnt_plan 2
purpose 14
addr_state 49
earliest_cr_line 668
initial_list_status 2
last_pymnt_d 13
last_credit_pull_d 14
application_type 2


In [13]:
loan_data['term'].value_counts(normalize = True)

term
 36 months    0.672691
 60 months    0.327309
Name: proportion, dtype: float64

In [14]:
# Convertimos la columna term a numérica
loan_data['term'] = pd.to_numeric(loan_data['term'].str.replace(' months', '') )

In [15]:
loan_data['term'].value_counts(normalize = True)

term
36    0.672691
60    0.327309
Name: proportion, dtype: float64

In [16]:
loan_data['emp_length'].value_counts(normalize = True)

emp_length
10+ years    0.356383
2 years      0.094394
< 1 year     0.086359
3 years      0.084077
1 year       0.069124
5 years      0.062799
4 years      0.061632
8 years      0.054245
7 years      0.046368
6 years      0.042381
9 years      0.042239
Name: proportion, dtype: float64

In [17]:
# Convertimos la columna emp_length a numérica

loan_data['emp_length'] = loan_data['emp_length'].str.replace('\+ years', '')
loan_data['emp_length'] = loan_data['emp_length'].str.replace('10+', '')
loan_data['emp_length'] = loan_data['emp_length'].str.replace('< 1 year', str(0))
loan_data['emp_length'] = loan_data['emp_length'].str.replace(' years', '')
loan_data['emp_length'] = loan_data['emp_length'].str.replace(' year', '')
loan_data['emp_length'] = pd.to_numeric(loan_data['emp_length'])
loan_data['emp_length'].fillna(value = 0, inplace = True)


In [18]:
loan_data['emp_length'].unique()

array([1., 7., 6., 0., 8., 9., 3., 4., 2., 5.])

In [19]:
loan_data['home_ownership'].value_counts(normalize = True)

home_ownership
MORTGAGE    0.493527
RENT        0.397761
OWN         0.108707
ANY         0.000005
Name: proportion, dtype: float64

In [20]:
loan_data['verification_status'].value_counts(normalize = True)

verification_status
Source Verified    0.426348
Verified           0.291626
Not Verified       0.282026
Name: proportion, dtype: float64

In [21]:
# vamos a transformar las columnas de fecha string fecha, a una diferencia de fechas en segundos
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

loan_data[date_cols].head(5)


,issue_d,earliest_cr_line,last_pymnt_d,last_credit_pull_d
0,Sep-15,Sep-00,NaN,Jan-16
1,Sep-15,Sep-02,Oct-15,Jan-16
2,Sep-15,Feb-06,Oct-15,Jan-16
3,Sep-15,Oct-99,Oct-15,Jan-16
4,Sep-15,Nov-99,Oct-15,Jan-16


In [22]:
# el formato origen en MES-YY

#función que crea una nueva columna con la diferencia en segundos entre una fecha auxiliar y la fecha de la columna
def date_columns_seconds(df, columns):
    for column in columns:
        # crea una fecha auxiliar para calcular la diferencia en segundos
        aux_date = pd.to_datetime('2023-09-24')
        # pasamos el string a formato fecha
        df[column] = pd.to_datetime(df[column], format = "%b-%y")
        # calculamos la diferencia en segundos
        df['seconds_since_' + column] = (aux_date - df[column]).dt.total_seconds()
        # reemplazamos los valores negativos por el máximo de la columna
        df['seconds_since_' + column] = df['seconds_since_' + column].apply(lambda x: df['seconds_since_' + column].max() if x < 0 else x)
    
date_columns_seconds(loan_data, date_cols)

loan_data[  [ 'seconds_since_'+ col  for col in date_cols] ].head(5)

,seconds_since_issue_d,seconds_since_earliest_cr_line,seconds_since_last_pymnt_d,seconds_since_last_credit_pull_d
0,254448000.0,727747200.0,NaN,243907200.0
1,254448000.0,664675200.0,251856000.0,243907200.0
2,254448000.0,556761600.0,251856000.0,243907200.0
3,254448000.0,756777600.0,251856000.0,243907200.0
4,254448000.0,754099200.0,251856000.0,243907200.0


In [23]:
#eliminamos las columnas de fecha para reducir dimensionalidad
loan_data.drop(columns = date_cols, inplace = True)

In [24]:
loan_data['purpose'].value_counts(normalize = True)

purpose
debt_consolidation    0.593315
credit_card           0.242831
home_improvement      0.060093
other                 0.045542
major_purchase        0.017675
medical               0.009364
car                   0.008235
small_business        0.007964
moving                0.005723
vacation              0.005328
house                 0.003395
renewable_energy      0.000524
wedding               0.000010
educational           0.000002
Name: proportion, dtype: float64

In [25]:
loan_data['addr_state'].value_counts(normalize = True)

addr_state
CA    0.137751
TX    0.082495
NY    0.080323
FL    0.069624
IL    0.040099
NJ    0.036094
PA    0.035527
OH    0.034228
GA    0.033503
NC    0.028546
VA    0.028530
MI    0.027195
MD    0.023857
AZ    0.023037
MA    0.022613
CO    0.021565
WA    0.021197
MN    0.018544
IN    0.017257
TN    0.016357
MO    0.015941
CT    0.014950
NV    0.014070
WI    0.013488
AL    0.012668
LA    0.012103
SC    0.012017
OR    0.011742
KY    0.009797
OK    0.009421
KS    0.008905
AR    0.007476
UT    0.006723
MS    0.006156
NM    0.005579
HI    0.004931
NH    0.004888
WV    0.004680
RI    0.004371
DE    0.002928
NE    0.002761
MT    0.002758
DC    0.002397
AK    0.002261
WY    0.002139
VT    0.002127
SD    0.001993
ME    0.001244
ND    0.001144
Name: proportion, dtype: float64

In [26]:
loan_data['pymnt_plan'].value_counts(normalize = True)

pymnt_plan
n    0.999998
y    0.000002
Name: proportion, dtype: float64

In [27]:
loan_data['initial_list_status'].value_counts(normalize = True)

initial_list_status
w    0.63523
f    0.36477
Name: proportion, dtype: float64

In [28]:
loan_data['application_type'].value_counts(normalize = True)

application_type
INDIVIDUAL    0.998785
JOINT         0.001215
Name: proportion, dtype: float64

### Segmentación de los datos

In [29]:
# vamos a separar los datos en prueba y entrenamiento

# Definir las características (X) y la variable objetivo (y)
X = loan_data.drop('target', axis=1)
y = loan_data['target']

# Separar los datos en conjuntos de entrenamiento y prueba

#stratify asegura que la distribución de la variable objetivo sea la misma en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

'''
Explicación de cada conjunto 

75% de los datos serán usados para el entrenamiento y 25% para la prueba

1. X_train: 
   - Contiene las características de los datos que se utilizarán para entrenar el modelo.
   - No incluye la variable objetivo, solo las características.

2. y_train: 
   - Contiene la variable objetivo correspondientes a X_train.
   - Se utiliza para entrenar el modelo junto con X_train, permitiendo que el modelo aprenda la relación entre las características y la variable objetivo.

3. X_test: 
   - Contiene las características de los datos que se utilizarán para evaluar el rendimiento del modelo una vez entrenado.
   - Al igual que X_train, solo contiene las características y no la variable objetivo.

4. y_test: 
   - Contiene la variable objetivo correspondientes a X_test.
   - Se utiliza para evaluar la precisión del modelo comparando las predicciones del modelo (basadas en X_test) con las verdaderas etiquetas en y_test.

'''


'\nExplicación de cada conjunto \n\n75% de los datos serán usados para el entrenamiento y 25% para la prueba\n\n1. X_train: \n   - Contiene las características de los datos que se utilizarán para entrenar el modelo.\n   - No incluye la variable objetivo, solo las características.\n\n2. y_train: \n   - Contiene la variable objetivo correspondientes a X_train.\n   - Se utiliza para entrenar el modelo junto con X_train, permitiendo que el modelo aprenda la relación entre las características y la variable objetivo.\n\n3. X_test: \n   - Contiene las características de los datos que se utilizarán para evaluar el rendimiento del modelo una vez entrenado.\n   - Al igual que X_train, solo contiene las características y no la variable objetivo.\n\n4. y_test: \n   - Contiene la variable objetivo correspondientes a X_test.\n   - Se utiliza para evaluar la precisión del modelo comparando las predicciones del modelo (basadas en X_test) con las verdaderas etiquetas en y_test.\n\n'